In [1]:
from sklearn.datasets import load_diabetes

import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

In [2]:
X,y = load_diabetes(return_X_y=True)

In [3]:
print(X.shape)
print(y.shape)

(442, 10)
(442,)


In [4]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=2)

In [5]:
reg = LinearRegression()
reg.fit(X_train,y_train)

LinearRegression()

In [6]:
print(reg.coef_)
print(reg.intercept_)

[  -9.15865318 -205.45432163  516.69374454  340.61999905 -895.5520019
  561.22067904  153.89310954  126.73139688  861.12700152   52.42112238]
151.88331005254167


In [7]:
y_pred = reg.predict(X_test)
r2_score(y_test,y_pred)

0.4399338661568968

In [8]:
import random

class MBGDRegressor:
    
    def __init__(self,batch_size,learning_rate=0.01,epochs=100):
        
        self.coef_ = None
        self.intercept_ = None
        self.lr = learning_rate
        self.epochs = epochs
        self.batch_size = batch_size
        
    def fit(self,X_train,y_train):
        # init your coefs
        self.intercept_ = 0
        self.coef_ = np.ones(X_train.shape[1])
        
        for i in range(self.epochs):
            
            for j in range(int(X_train.shape[0]/self.batch_size)):              # no. of updates per epoch = total_samples / batch_size
                
                idx = random.sample(range(X_train.shape[0]),self.batch_size)    # random index of batch_size no. of rows
                
                y_hat = np.dot(X_train[idx],self.coef_) + self.intercept_       # y_pred for batch_size no. of rows (note that fancy indexing is used) 
                #print("Shape of y_hat",y_hat.shape)
                intercept_der = -2 * np.mean(y_train[idx] - y_hat)
                self.intercept_ = self.intercept_ - (self.lr * intercept_der)

                coef_der = -2 * np.dot((y_train[idx] - y_hat),X_train[idx])
                self.coef_ = self.coef_ - (self.lr * coef_der)
        
        print(self.intercept_,self.coef_)
    
    def predict(self,X_test):
        return np.dot(X_test,self.coef_) + self.intercept_

In [9]:
mbr = MBGDRegressor(batch_size=int(X_train.shape[0]/50),learning_rate=0.01,epochs=100)

In [10]:
mbr.fit(X_train,y_train)

152.33393499310353 [  23.14018307 -146.44865976  444.81032442  302.36149264  -18.39560717
  -95.46637123 -190.27749461  115.69143765  418.73239932  103.96514457]


In [11]:
y_pred = mbr.predict(X_test)

In [12]:
r2_score(y_test,y_pred)

0.453750561648

## For sklearn there is no Mini Batch GD so we have to do a workaround

In [13]:
from sklearn.linear_model import SGDRegressor

sgd = SGDRegressor(learning_rate='constant',eta0=0.1)    # no need for max iter as we are gonna use partial fit
batch_size = 35

# total there will be 100 updates, instead of dividing into epochs and all
# each update will look at 35 samples 
for i in range(100):   
    
    idx = random.sample(range(X_train.shape[0]),batch_size)      # random index of batch_size no. of samples
    sgd.partial_fit(X_train[idx],y_train[idx])                   # partial fit method only does one epoch (one update) of GD for n samples

In [14]:
sgd.coef_

array([  50.69655645,  -57.214631  ,  346.39083638,  221.00836246,
         19.30039674,  -20.49941829, -172.16142894,  133.65580258,
        310.01947691,  128.2436516 ])

In [15]:
sgd.intercept_

array([140.96075663])

In [16]:
y_pred = sgd.predict(X_test)

In [17]:
r2_score(y_test,y_pred)

0.40900183175487625

In [18]:
sgd.intercept_ - mbr.intercept_, sgd.coef_ - mbr.coef_

(array([-11.37317836]),
 array([  27.55637338,   89.23402875,  -98.41948804,  -81.35313017,
          37.69600391,   74.96695295,   18.11606567,   17.96436493,
        -108.71292242,   24.27850703]))